In [ ]:
from unsloth import FastModel
import torch
import torch.nn as nn
model, tokenizer = FastModel.from_pretrained(
    # model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    model_name = "unsloth/Qwen3-8B-Base-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    max_seq_length = 8192, # Choose any for long context!
    load_in_4bit = True,
    # resize_model_vocab=2,
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1624556268.py, line 6)

In [2]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096, padding_idx=151654)
    (layers): ModuleList(
      (0-2): 3 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06)
  

In [ ]:
# save the lm_head for yes and no
index = [tokenizer.encode(" No")[0], tokenizer.encode(" Yes")[0]]
print(index)
model.lm_head.weight[index]
custom_embedding = nn.Embedding(2, 4096)
custom_embedding.weight.data = model.lm_head.weight.data[index]
torch.save(custom_embedding.state_dict(), './Model/Gwen8B_lm_head.pth')

[2308, 7414]


torch.Size([2, 4096])

In [9]:
tokenizer.encode??

Signature:
tokenizer.encode(
    text: Union[str, list[str], list[int]],
    text_pair: Union[str, list[str], list[int], NoneType] = None,
    add_special_tokens: bool = True,
    padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = False,
    truncation: Union[bool, str, transformers.tokenization_utils_base.TruncationStrategy, NoneType] = None,
    max_length: Optional[int] = None,
    stride: int = 0,
    padding_side: Optional[str] = None,
    return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None,
    **kwargs,
) -> list[int]
Docstring:
Converts a string to a sequence of ids (integer), using the tokenizer and vocabulary.

Same as doing `self.convert_tokens_to_ids(self.tokenize(text))`.

Args:
    text (`str`, `list[str]` or `list[int]`):
        The first sequence to be encoded. This can be a string, a list of strings (tokenized string using the
        `tokenize` method) or a list of integers (tokenized string ids using the `convert_t

In [3]:
tokenizer.encode(
            "Violation:", add_special_tokens=False
        )

[74971, 25]

In [14]:
subreddit = "test"
rule = "test"
comment1 = "test"
label1 = "test"


tokenizer.encode(
            "You are given a comment on reddit. Your task is to classify if it violates the given rule.\n"
            f"Subreddit: r/{subreddit}\n"
            f"Rule: {rule}\n"
            f"Comment: {comment1}\n"
            f"Violation: {label1}\n"
            f"Comment: {comment1}\n"
            f"Violation: {label1}\n"
            , add_special_tokens=True,
            return_tensors="pt",
        )

tensor([[ 2610,   525,  2661,   264,  3980,   389, 42174,    13,  4615,  3383,
           374,   311, 48129,   421,   432, 50313,   279,  2661,  5912,   624,
          3136, 36933,    25,   435, 12697,   198, 11337,    25,  1273,   198,
         10677,    25,  1273,   198, 74971,    25,  1273,   198, 10677,    25,
          1273,   198, 74971,    25,  1273,   198]])

In [39]:
import importlib
from utility import build_dataloader

# After making changes to the utility module
import utility
importlib.reload(utility)
from utility import build_dataloader  # Get the updated version

In [ ]:
from utility import build_dataloader
import pandas as pd
df = pd.read_csv("Data/Data1/train.csv")
dataloader = build_dataloader(df, tokenizer)
for row_id, input_ids, vi_index, labels in dataloader:
    row_id = int(row_id)
    input_ids, vi_index, labels = input_ids.to('cuda'), vi_index.to('cuda'), labels.to('cuda')
    break

In [45]:
int(row_id)

1116

In [42]:
vi_index

tensor([ 79, 121, 165])

In [43]:
input_ids[0, vi_index]

tensor([25, 25, 25])

In [44]:
labels

tensor([1, 0])

In [46]:
input_ids

tensor([[ 2610,   525,  2661,   264,  3980,   389, 42174,    13,  4615,  3383,
           374,   311, 48129,   421,   432, 50313,   279,  2661,  5912,   624,
          3136, 36933,    25,   435,  2687, 96188, 60975,   198, 11337,    25,
          2308, 46323,    25, 81667,    11, 44780,  7746,    11,  6975, 91104,
         12876,    11,   323, 34282,  2213,   525,   537,  5420,   624, 10677,
            25, 27553, 10546,  2611,    11,   600,  1184,  5274,  1431,   421,
           575,  1366,  3736,   847, 20036,  6548,   600,  3783,   498,  1588,
           366,  3144, 32005,   454,    16,    17,   905,   397, 74971,    25,
          7414,   198, 10677,    25,  8030,  9203,   481,   508, 22574,  9203,
          9533,    27,  3144, 12673,    17,    19, 83560,   905,    29,   760,
            58,  9951,    45,  9203,   220,    17,  9533,    27,  3144, 12673,
            17,    19, 83560,   905,   397, 18370,   481,  2806, 66265,   198,
         74971,    25,  2308,   198, 10677,    25,  